In [26]:
%load_ext autoreload 

import numpy as np
import pandas as pd
# from aocd i
# import get_data
import os
import re
import time
import networkx as nx
from heapq import *
import functools
import functools
from  math import log
from tqdm import tqdm


module_path=os.path.abspath(os.path.join('../'))
sys.path.append(module_path)
from ComplexMap import ComplexMap

file="test.txt"
# file="test2.txt"
# file ="input.txt"


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
def getSpecialEdges3(G,area,src,tgt,skip_dist=2,threshold=100,to_beat=None):
    to_check = set()
    
    target_lengths = nx.shortest_path_length(G,target=tgt)
    source_lengths = nx.shortest_path_length(G,source=src)
        
    for n1 in tqdm(source_lengths):
        for n2 in target_lengths:
            d=dist(n1,n2)
            if d <= 1 or d > skip_dist: 
                continue
            
            if d + source_lengths[n1]+target_lengths[n2] <= to_beat - threshold:
                if (n2,n1,d) not in to_check: to_check.add((n1,n2,d)) 
    print(f"to_check={len(to_check)}")
    return to_check

In [28]:
def getCheats(G,src,tgt,cheats,to_beat=np.Inf,threshold=1):
    res = 0
    res_dict = {}
    
    for c1,c2,w in tqdm(cheats):
        G.add_edge(c1,c2,weight=w)
        
        path_len = nx.shortest_path_length(G,src,tgt, weight="weight")
        gain = to_beat-path_len
        if gain>= threshold:
            res+=1
        
        G.remove_edge(c1,c2)
    
    return res


In [29]:
def constructAreaAndGraph(file):
    with open(file) as f:
        lines = f.readlines()

    lines = list(map(lambda x:x.strip(),lines))
    area = ComplexMap(lines,str)

    
    edge_list = area.get_edges_for_graph(vals_to_consider=[".","S","E"],includes_given=True)
    
    edge_list = {(c1,c2,1) for c1,c2 in edge_list}
    G=nx.Graph()
    for p1,p2,w in edge_list: G.add_edge(p1,p2,weight=w)

    return area,G

In [30]:
def dist(c1,c2):
    d=c2-c1
    return int(abs(d.real)+abs(d.imag))

In [41]:
def partA(file,area=None,G=None,special_rules = False,threshold=1,skip_dist=2):

    if area is None or G is None: 
        area,G=constructAreaAndGraph(file)
        
    area_as_set = area.map_as_set()
    src = [coord for coord,val in area_as_set if val == "S"][0]
    tgt = [coord for coord,val in area_as_set if val == "E"][0]
      
    to_beat = nx.shortest_path_length(G,src,tgt,weight="weight")
    
    print("to_beat",to_beat)

    # get special edges
    print("pre special edges")
    special_edges = getSpecialEdges3(G,area,src,tgt,skip_dist=skip_dist,to_beat=to_beat,threshold=threshold)
    print("post special edges")

    numCheats = len(special_edges)
    
    return numCheats, G, area

IndentationError: unindent does not match any outer indentation level (<string>, line 20)

In [32]:
res,_,_=partA("test.txt",True,threshold=1,skip_dist=2)
print(res)
assert res == 44
print("OK")


to_beat 84
pre special edges


100%|███████████████████████████████████████████████████████████████████████████████| 85/85 [00:00<00:00, 21270.56it/s]

to_check=44
post special edges
44
OK


In [33]:
res,_,_=partA(file,threshold=50,skip_dist=20)
print(res)
assert res == 285
print("OK")


to_beat 84
pre special edges


100%|███████████████████████████████████████████████████████████████████████████████| 85/85 [00:00<00:00, 10627.67it/s]

to_check=285
post special edges
285
OK


In [34]:
area_big,G_big = constructAreaAndGraph("input.txt")

In [35]:
area_small,G_small = constructAreaAndGraph("test.txt")

In [36]:
# sh_paths_dict_big = getShortestPaths(G_big)

In [37]:
G_big_copy = G_big.copy()
num_cheats,_,_ =partA("input.txt",area_big,G_big_copy,threshold=100,skip_dist=2)
assert num_cheats == 1286
print("OK")

to_beat 9416
pre special edges


100%|█████████████████████████████████████████████████████████████████████████████| 9417/9417 [00:45<00:00, 204.91it/s]

to_check=1286
post special edges
OK


In [39]:
G_big_copy = G_big.copy()
num_cheats,_,_ =partA("input.txt",area_big,G_big_copy,threshold=100,skip_dist=20)
assert num_cheats == 989316
print("OK")
print(num_cheats)

to_beat 9416
pre special edges


100%|█████████████████████████████████████████████████████████████████████████████| 9417/9417 [00:49<00:00, 191.93it/s]


to_check=989316
post special edges
OK
989316
